# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/3fwVL/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic(1)
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree(1)
          

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120008e+02     1.517772e+00
 * time: 0.6591520309448242
     1     1.122989e+01     8.425008e-01
 * time: 1.731323003768921
     2    -1.148855e+01     9.736868e-01
 * time: 2.224203109741211
     3    -3.421497e+01     8.008162e-01
 * time: 2.9579901695251465
     4    -4.794966e+01     5.308069e-01
 * time: 3.702601194381714
     5    -5.693354e+01     1.836627e-01
 * time: 4.418258190155029
     6    -5.976900e+01     9.083519e-02
 * time: 4.903770208358765
     7    -6.076600e+01     7.146675e-02
 * time: 5.3871870040893555
     8    -6.123562e+01     6.610673e-02
 * time: 5.89021110534668
     9    -6.156744e+01     3.598846e-02
 * time: 6.372791051864624
    10    -6.179919e+01     2.442849e-02
 * time: 6.855967998504639
    11    -6.194350e+01     2.335409e-02
 * time: 7.33915901184082
    12    -6.203032e+01     2.160735e-02
 * time: 7.833080053329468
    13    -6.209222e+01     1.692527e-02
 * time: 8.315371990203857
    14

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671065
    AtomicLocal         -18.8557650
    AtomicNonlocal      14.8522626
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485362 
    Xc                  -19.3336816

    total               -62.261666456038